# Deep Learning: Image Classification

##### Completed by: Jessica Khaskheli

##### Contributors: None

##### Date: May 31, 2022


In this project, I will be using TensorFlow, ResNet50 and Keras and the viziometrics images dataset to load the images, understand the parameterization of the generators, and extract features to train a neural network on its image classification layers.


##### Loading the Data

In [9]:
import numpy as np
import keras
from keras.preprocessing.image import ImageDataGenerator
#from keras.applications.resnet50 import ResNet50, decode_predictions
#from keras.applications.resnet50 
from tensorflow.keras.applications.resnet50 import ResNet50, decode_predictions
from keras import layers
from keras.models import Model, Sequential
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [12]:
# importing required modules
from zipfile import ZipFile
  
# specifying the zip file name
file_name = "viziometrics.zip"
  
# opening the zip file in READ mode
with ZipFile(file_name, 'r') as zip:
    # printing all the contents of the zip file
    zip.printdir()
  
    # extracting all the files
    #print('Extracting all the files now...')
    zip.extractall()
print('done')

File Name                                             Modified             Size
viziometrics/                                  2019-11-04 18:53:54            0
__MACOSX/._viziometrics                        2019-11-04 18:53:54          311
viziometrics/test/                             2018-04-01 16:54:48            0
__MACOSX/viziometrics/._test                   2018-04-01 16:54:48          211
viziometrics/train/                            2018-04-01 16:54:48            0
__MACOSX/viziometrics/._train                  2018-04-01 16:54:48          211
viziometrics/val/                              2018-04-01 16:54:48            0
__MACOSX/viziometrics/._val                    2018-04-01 16:54:48          211
viziometrics/test/scheme/                      2018-04-01 16:54:48            0
__MACOSX/viziometrics/test/._scheme            2018-04-01 16:54:48          211
viziometrics/test/visualization/               2018-04-01 16:54:48            0
__MACOSX/viziometrics/test/._visualizati

In [13]:
# create train, validation, and test generators from our image directory

datagen = ImageDataGenerator()

train_generator = datagen.flow_from_directory(
  directory=r"./viziometrics/train/",
  target_size=(224, 224),
  color_mode="rgb",
  batch_size=32,
  class_mode="categorical",
  shuffle=False,
  seed=42
)

val_generator = datagen.flow_from_directory(
  directory=r"./viziometrics/val/",
  target_size=(224, 224),
  color_mode="rgb",
  batch_size=32,
  class_mode="categorical",
  shuffle=False,
  seed=42
)

test_generator = datagen.flow_from_directory(
  directory=r"./viziometrics/test/",
  target_size=(224, 224),
  color_mode="rgb",
  batch_size=1,
  class_mode=None,
  shuffle=False,
  seed=42
)

Found 2733 images belonging to 5 classes.
Found 1571 images belonging to 5 classes.
Found 1563 images belonging to 5 classes.


##### 1. Extracting Features

I will use a pretrained network (ResNet50) to extract features from the images. Then we will train a neural network on the features and labels.

In [15]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [96]:
# download the pre-trained ResNet50 model
resnet = keras.applications.resnet.ResNet50(weights='imagenet', input_shape=(224, 224, 3))

# inspect the ResNet50 architecture
resnet.summary()


Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_3[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [104]:
# I want the outputs from the 'conv5_block_3_out' layer

layer_name = 'conv5_block3_out'
resnet_embedder = Model(inputs=resnet.input,outputs=resnet.get_layer(layer_name).output)

# saving the layer's shape for later use
DIM = 2048

##### Using ResNet50 to extract features from the images

Using the `model.predict()` function will help me extract features. This will return the output of the last layer of ResNet50, namely, conv5_block3_out (Activation). These are the features captured by the pretrained network.

In [123]:
# use ResNet50 to extract features from the images
train_features = resnet_embedder.predict(train_generator, verbose=1) # takes ~4min on my laptop

86/86 [==============================] - 324s 4s/step


In [109]:
# validation features
val_features = resnet_embedder.predict(val_generator, verbose = 1)

50/50 [==============================] - 182s 4s/step


In [118]:
# test features
test_features = resnet_embedder.predict(test_generator, verbose=1, batch_size = 1)

1563/1563 [==============================] - 249s 158ms/step


In [124]:
# printing shapes of all three features
print(train_features.shape, val_features.shape, test_features.shape)

(2733, 7, 7, 2048) (1571, 7, 7, 2048) (1563, 7, 7, 2048)


I want to reshape the features into 2D arrays, which will be the input X of the neural network classifier.

In [125]:
# reshape the features to 2D arrays
train_X = train_features.reshape((2733, -1))
val_X = val_features.reshape((1571, -1))
test_X = test_features.reshape((1563, -1))

print(train_X.shape, val_X.shape, test_X.shape)

(2733, 100352) (1571, 100352) (1563, 100352)


Reading the class labels from the generators.

In [132]:
# # read the class labels from the generators
train_labels = train_generator.classes
val_labels = val_generator.classes
test_labels = test_generator.classes

print(test_labels)

[0 0 0 ... 4 4 4]


I plan to one-hot encode the labels and they will be the "y" that the model will predict.

In [214]:
# # get one-hot encoding of labels
def get_one_hot(labels, nb_classes):
     res = np.eye(nb_classes)[np.array(labels).reshape(-1)]
     return res.reshape(list(labels.shape)+[nb_classes])

# # use get_one_hot()
NUM_CLASSES = 5
train_y = get_one_hot(train_labels, NUM_CLASSES)
val_y = get_one_hot(val_labels, NUM_CLASSES)
test_y = get_one_hot(test_labels, NUM_CLASSES)

In [215]:
# printing shape of train, test, and validation sets
train_y.shape
val_y.shape
test_y.shape

(1563, 5)

I plan to define a neural network that contains a few layers.

In [261]:
# create neural network model
model = Sequential()

model.add(layers.Dense(200, activation = "relu", input_shape=(100352,))) 
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(NUM_CLASSES, activation = "softmax"))
# print out network architecture
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 200)               20070600  
                                                                 
 dropout_12 (Dropout)        (None, 200)               0         
                                                                 
 dense_21 (Dense)            (None, 5)                 1005      
                                                                 
Total params: 20,071,605
Trainable params: 20,071,605
Non-trainable params: 0
_________________________________________________________________


##### Compiling model

In [1]:
# compiling the model using `sgd` before training
model.compile(optimizer = 'sgd', loss = ['categorical_crossentropy'], metrics = ['accuracy'])

NameError: name 'model' is not defined

I plan to train the model for about 20 epochs, or until the validation accuracy reaches 0.8. (whichever comes earlier). The model will be trained on `train_X` and `train_y` and will be validated on `val_X` and `val_y`.

In [263]:
import tensorflow
checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

# training the model for about 20 epochs
model.fit(x = train_X, y = train_y, validation_data = (val_X, val_y), epochs = 20, callbacks=model_checkpoint_callback)

Epoch 1/20
86/86 [==============================] - 10s 109ms/step - loss: 2.1788 - accuracy: 0.7955 - val_loss: 1.3773 - val_accuracy: 0.5245
Epoch 2/20
86/86 [==============================] - 10s 111ms/step - loss: 0.2888 - accuracy: 0.9038 - val_loss: 1.0511 - val_accuracy: 0.6645
Epoch 3/20
86/86 [==============================] - 8s 94ms/step - loss: 0.2086 - accuracy: 0.9330 - val_loss: 0.9343 - val_accuracy: 0.6760
Epoch 4/20
86/86 [==============================] - 7s 82ms/step - loss: 0.1821 - accuracy: 0.9374 - val_loss: 0.9529 - val_accuracy: 0.6703
Epoch 5/20
86/86 [==============================] - 8s 97ms/step - loss: 0.1561 - accuracy: 0.9473 - val_loss: 1.0317 - val_accuracy: 0.7027
Epoch 6/20
86/86 [==============================] - 9s 102ms/step - loss: 0.1188 - accuracy: 0.9616 - val_loss: 0.8593 - val_accuracy: 0.7588
Epoch 7/20
86/86 [==============================] - 8s 88ms/step - loss: 0.1025 - accuracy: 0.9663 - val_loss: 0.8386 - val_accuracy: 0.7607
Epoch 8/

With this model, the best training accuracy I observed was 0.9920 in the 19th epoch while the best validation accuracy I observed was 0.8447 in the 17th epoch.

##### Reporting the Test Accuracy

In [265]:
# loading the weights of best model
model.load_weights(checkpoint_filepath)

# measuring test accuracy
model.fit(x = test_X, y = test_y, validation_data = (val_X, val_y), epochs = 20, callbacks=model_checkpoint_callback)

Epoch 1/20
49/49 [==============================] - 8s 148ms/step - loss: 0.6550 - accuracy: 0.7742 - val_loss: 0.6317 - val_accuracy: 0.7581
Epoch 2/20
49/49 [==============================] - 6s 125ms/step - loss: 0.5104 - accuracy: 0.8119 - val_loss: 0.3682 - val_accuracy: 0.8797
Epoch 3/20
49/49 [==============================] - 5s 100ms/step - loss: 0.3899 - accuracy: 0.8567 - val_loss: 0.7293 - val_accuracy: 0.7263
Epoch 4/20
49/49 [==============================] - 6s 123ms/step - loss: 0.2966 - accuracy: 0.8868 - val_loss: 0.3700 - val_accuracy: 0.8835
Epoch 5/20
49/49 [==============================] - 6s 116ms/step - loss: 0.2596 - accuracy: 0.8938 - val_loss: 0.3560 - val_accuracy: 0.8867
Epoch 6/20
49/49 [==============================] - 5s 101ms/step - loss: 0.2518 - accuracy: 0.9123 - val_loss: 0.4133 - val_accuracy: 0.8498
Epoch 7/20
49/49 [==============================] - 4s 89ms/step - loss: 0.2443 - accuracy: 0.9143 - val_loss: 0.4252 - val_accuracy: 0.8657
Epoch 8

The best test accuracy I was able to observe was 0.9776 in the 20th epoch when training my model on the classification layers.